In [1]:
import torch 
import torch.nn as nn
from d2l import torch as d2l

6.2.1 交叉相关 计算

In [6]:
def corr2d(X, K):
    h, w = K.shape
    Y = torch.zeros((X.shape[0]  - h +1, X.shape[1] - w +1))

    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i: i+h, j:j+w] * K).sum()
    return Y

In [7]:
X = torch.tensor([[1, 2, 3], [4,5,6], [7,8,9]])
K = torch.tensor([[0,1], [2,3]])
corr2d(X, K)

tensor([[25., 31.],
        [43., 49.]])

6.2.2 卷积层

In [8]:
class Conv2d(nn.Module):
    def __init__(self, kernel_size):
        super().__init__()
        self.weight = nn.Parameter(torch.rand(kernel_size))
        self.bias = nn.Parameter(torch.zeros(1))

    def forward(self, x):
        return corr2d(x, self.weight) + self.bias


6.2.3 图像中目标的边缘检测

In [9]:
x = torch.ones((6, 8))
x[:, 2:6] = 0
x

tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.],
        [1., 1., 0., 0., 0., 0., 1., 1.]])

In [12]:
kernel = torch.tensor([[1,-1]])

Y = corr2d(x, kernel)
Y

tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
        [ 0.,  1.,  0.,  0.,  0., -1.,  0.]])

In [17]:
corr2d(x.T, kernel)

tensor([[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]])

In [52]:
conv2d = nn.Conv2d(1, 1, (1, 2), bias = False)
X = x. reshape((1, 1, 6, 8))
Y = Y.reshape((1, 1, 6, 7))

lr = 0.03

for i in range(10):
    Y_hat = conv2d(X)
    l = (Y_hat - Y) ** 2
    conv2d.zero_grad()
    l.sum().backward()

    conv2d.weight.data[:] -= lr * conv2d.weight.grad

    if (i+1)%2==0:
            print(f'epoch   {i+1} , loss   {l.sum():.3f}')

epoch   2 , loss   5.336
epoch   4 , loss   1.110
epoch   6 , loss   0.274
epoch   8 , loss   0.082
epoch   10 , loss   0.029


In [59]:
conv2d.weight.data.reshape((1,2))

tensor([[ 0.9727, -1.0054]])